In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np

## Γεωμετρικές σταθερές WGS84 
wgs84_a = 6378137e0
wgs84_f = 1/298.257223563

def semi_minor(semi_major, flattening):
    """ Υολογισμός μικρού ημιάξονα β = α * (1-f)
        
        Παράμετροι:
        ----------
        semi_major: Μεγάλος ημιάξονας ελλειψοειδούς α, σε [m].
        flattening: Επιπλάτυνση [-]
        
        Εξαγώμενα:
        -----------
        Μικρός ημιάξονας ελλειψοειδούς β, σε [m].
    """
    return semi_major*(1e0-flattening)

def eccentricity(flattening):
    """ Υολογισμός εκκεντρότητας, ε = (f * (2-f))^(1/2)
        
        Παράμετροι:
        ----------
        flattening: Επιπλάτυνση ελλειψοειδούς [-]
        
        Εξαγώμενα:
        -----------
        Εκκεντρότητα ελλειψοειδούς, ε.
    """
    return math.sqrt(flattening * (2e0 - flattening))

def N(semi_major, eccentricity, geodetic_latitude):
    """ Ακτίνα καμπυλότητας της κύριας κάθετης τομής Ν [m]
        Ν = α / (1-ε^2 * sin^2f)^(1/2)
        
        Παράμετροι:
        ----------
        semi_major: Μεγάλος ημιάξονας ελλειψοειδούς α, σε [m].
        eccentricity: Εκκεντρότητα [-].
        geodetic_latitude: Γεωδαιτικό πλάτος φ, σε [rad].
        
        Εξαγώμενα:
        -----------
        Ακτίνα καμπυλότητας της κύριας κάθετης τομής Ν σε [m].
    """
    sf = math.sin(geodetic_latitude);
    sf2 = sf * sf
    e2 = eccentricity * eccentricity
    return semi_major / math.sqrt(1e0 - e2 * sf2)

In [45]:
def decd2hexd(decimal_deg):
    """ Δεκαδικές μοίρες, σε μοίρες, λεπτά, δεύτερα και πρόσημο.
        
        Παράμετροι:
        ----------
        decimal_deg: Δεκαδικές μοίρες.
        
        Εξαγώμενα:
        -----------
        [deg, min, sec, sign], με
        deg: Μοίρες, πάντα θετικές, ακέραιες
        min: Λεπτά της μοίρας, θετικά, ακέραια
        sec: Δεύτερα της μοίρας, θετικά, πραγματικός αριθμός
        sign: Υποδυκνύει θετική ή αρνητική γωνία
    """
    decdeg = abs(decimal_deg)
    deg = int(decdeg)
    min = int((decdeg - float(deg)) * 60e0)
    sec = decdeg - (float(deg) + float(min) / 60e0)
    sec *= 3600e0
    sign = int(math.copysign(1e0, decimal_deg))
    return deg, min, sec, sign

class CartesianSet3D:
    """ Κλάση για αναπαράσταση ενός σημείου σε 3Δ καρτεσιανές συν/νες. """
    def __init__(self,x,y,z):
        self.x = x
        self.y = y
        self.z = z

class GeodeticSet3D:
    """ Κλάση για ναπαράσταση ενός σημείου σε 3Δ γεωδαιτικές συν/νες. """
    def __init__(self,latitude,longitude,height):
        self.latitude = latitude
        self.longitude = longitude
        self.height = height

def geodetic2cartesian(semi_major, flattening, geo):
    """ Γεωδαιτικές σε καρτεσιανές συν/νες.
       
        Παράμετροι:
        ----------
        semi_major: Μεγάλος ημιάξονας ελλειψοειδούς α, σε [m].
        flattening: Επιπλάτυνση [-]
        geo: Γεωδαιτικές συν/νες ως GeodeticSet3D.
        
        Εξαγώμενα:
        -----------
        Καρτεσιανές συν/νες του geo ως CartesianSet3D
    """
    e = eccentricity(flattening)
    Nf = N(semi_major, e, geo.latitude)

    x = (Nf + geo.height) * math.cos(geo.latitude) * math.cos(geo.longitude)
    y = (Nf + geo.height) * math.cos(geo.latitude) * math.sin(geo.longitude)
    z = (Nf * (1e0 - e * e) + geo.height) * math.sin(geo.latitude)
    return CartesianSet3D(x,y,z)

def cartesian2geodetic(semi_major, flattening, crt, convergence_limit_arcsec):
    """ Καρτεσιανές σε γεωδαιτικές συν/νες.
       
        Παράμετροι:
        ----------
        semi_major: Μεγάλος ημιάξονας ελλειψοειδούς α, σε [m].
        flattening: Επιπλάτυνση [-]
        crt: Καρτεσιανές συν/νες ως CartesianSet3D
        convergence_limit_arcsec: Κριτήριο σύγκλισης για το πλάτος, σε [arcsec]
        
        Εξαγώμενα:
        -----------
        Γεωδαιτικές συν/νες του crt ως GeodeticSet3D
    """
    e = eccentricity(flattening)
    MAX_ALLOWED_ITERATIONS = 100

    # convergernce criterion from sec to rad
    max_diff = convergence_limit_arcsec * (1e0 / 3600e0) * math.pi / 180e0

    # longitude in range [-180,180]
    longitude = math.atan2(crt.y, crt.x)

    # initial value for latitude
    rho = math.sqrt(crt.x * crt.x + crt.y * crt.y)
    latip1 = math.atan(crt.z / ((1e0 - e * e) * rho))
    
    # loop untill precision is met, counting iterations
    lati = -1e20
    it = 0
    while abs(lati - latip1) > max_diff and it < MAX_ALLOWED_ITERATIONS:
        lati = latip1
        Ni = N(semi_major, e, lati)
        latip1 = math.atan((crt.z + e * e * Ni * math.sin(lati)) / rho)
        it += 1
        print("Iteration {:} lat_old = {:.12f} lat_new = {:.12f} diff {:.6f} [arcsec]".format(it, np.degrees(lati), np.degrees(latip1),  (np.degrees(lati) - np.degrees(latip1))*3600e0))

    # check for non-convergence and throw if error
    if it >= MAX_ALLOWED_ITERATIONS:
        print("Max iterations reached without convergence!")
        raise RuntimeError("Failed to transform cartesian to geodetic coordinates")

    # compute ellipsoidal height
    height = rho / math.cos(latip1) - N(semi_major, e, latip1)

    return GeodeticSet3D(latip1, longitude, height)


In [46]:
## Validation check, Step 1:
## Transform geodetic to cartesian coordinates
latitude = 37 + 58/60e0 + 30.9/3600e0
longitude = 23 + 46/60e0 + 47.0/3600e0
height = 237.904
geo = GeodeticSet3D(math.radians(latitude), math.radians(longitude), height)
crt = geodetic2cartesian(6378137.0, 1/298.257223563, geo)
print("Step 1: {:12.3f} {:12.3f} {:12.3f}".format(crt.x, crt.y, crt.z))

## Validation check, Step 2:
## Transform back from cartesian to geodetic
geo2 = cartesian2geodetic(6378137.0, 1/298.257223563, crt, 1e-5)
fdeg, fmin, fsec, fsign = decd2hexd(math.degrees(geo2.latitude))
ldeg, lmin, lsec, lsign = decd2hexd(math.degrees(geo2.longitude))
print("Step 2: {:+02d} {:02d} {:7.4f} {:+02d} {:02d} {:7.4f} {:10.3f}".format(fsign*fdeg, fmin, fsec, lsign*ldeg, lmin, lsec, geo2.height))

## Difference between geo and geo2 in arcseconds/meters
latitude_diff = geo.latitude - geo2.latitude
longitude_diff = geo.longitude - geo2.longitude
height_diff = geo.height - geo2.height
print("Diffs : {:+12.5f} {:+12.5f} {:+12.5f}".format(latitude_diff, longitude_diff, height_diff))

Step 1:  4606907.220  2029940.874  3903425.201
Iteration 1 lat_old = 37.975256977038 lat_new = 37.975250029095 diff 0.025013 [arcsec]
Iteration 2 lat_old = 37.975250029095 lat_new = 37.975250000121 diff 0.000104 [arcsec]
Iteration 3 lat_old = 37.975250000121 lat_new = 37.975250000000 diff 0.000000 [arcsec]
Step 2: +37 58 30.9000 +23 46 47.0000    237.904
Diffs :     -0.00000     +0.00000     -0.00000


In [49]:
# δφ σε (μοίρες, λεπτά δεύτερα)
delta_lats = [(1,0,0.), (0,1,0.), (0,0,1.), (0,0,.1), (0,0,.01), (0,0,.001), (0,0,.0001), (0,0,.00001)]
for df in delta_lats:
    # δφ σε rad
    df_rad = np.radians(df[0] + df[1]/60. + df[2]/(60.*60.))
    s = df_rad * 6378137.
    print("δφ = {:d}° {:2d}' {:5.5f}\" -> δs = {:12.3f} [m]".format(df[0], df[1], df[2], s))

δφ = 1°  0' 0.00000" -> δs =   111319.491 [m]
δφ = 0°  1' 0.00000" -> δs =     1855.325 [m]
δφ = 0°  0' 1.00000" -> δs =       30.922 [m]
δφ = 0°  0' 0.10000" -> δs =        3.092 [m]
δφ = 0°  0' 0.01000" -> δs =        0.309 [m]
δφ = 0°  0' 0.00100" -> δs =        0.031 [m]
δφ = 0°  0' 0.00010" -> δs =        0.003 [m]
δφ = 0°  0' 0.00001" -> δs =        0.000 [m]


In [41]:
num_tests = 100
# κριτήριο σύγκλισης, 0.1 [arcsec] σε [rad]
TOLERANCE = np.radians(.1 / 3600.)
for test_nr in range(num_tests):
    # τυχαίο πλάτος
    lat = np.random.uniform(-90e0, 90e0)
    # τυχαίο μήκος
    lon = np.random.uniform(-180e0, 180e0)
    # τυχαίο υψόμετρο
    h = np.random.uniform(0, 1e3)
    # γεωδαιτικές σε καρτεσιανές ...
    crt = geodetic2cartesian(wgs84_a, wgs84_f, GeodeticSet3D(np.radians(lat), np.radians(lon), h))
    # καρτεσιανές σε  γεωδαιτικές ...
    geo = cartesian2geodetic(wgs84_a, wgs84_f, crt, 1e-3)
    # επαλήθευση
    assert( abs(geo.latitude-np.radians(lat))<TOLERANCE )
    assert( abs(geo.longitude-np.radians(lon))<TOLERANCE )
    assert( abs(geo.height-h)<1e-3 )